# Q6 - ELM Classifier

In [2]:
import numpy as np
import pandas as pd
import scipy.io
import random
from sklearn.metrics import confusion_matrix,accuracy_score
pd.set_option('display.max_rows', 5100)
pd.set_option('display.max_columns', 5010)
pd.set_option('display.width', 10100)
import seaborn as sns

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data=scipy.io.loadmat('/content/drive/MyDrive/2018A8PS1229H_NNFL/Data/data5.mat')
data=pd.DataFrame(data['x'])

In [5]:
col_names=[str(x) for x in range(72)]
col_names.append('target')
data.columns=col_names
X_orig=data.iloc[:,:-1]
y_orig=data.iloc[:,-1]

In [6]:
def normalize(X):
    return (X-X.mean())/X.std()

In [7]:
X=normalize(X_orig)
y=y_orig

In [8]:
class ELM:
    def __init__(self,l=10):
        self.l=l
        
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))
    
    def train(self,X,y,kind=None):
        self.kind=kind
        if kind=="sigmoid":
            self.a=2*np.random.randn(X.shape[1],self.l)-1
            self.b=2*np.random.randn(X.shape[0],1)-1
            f_x=X@self.a+self.b
            h=1/(1+np.exp(-f_x))
        elif kind=="tanh":
            self.a=np.random.randn(X.shape[1],self.l)
            self.b=np.random.randn(X.shape[0],1)
            f_x=X@self.a+self.b
            h=(1-np.exp(-f_x))/(1+np.exp(-f_x))
        elif kind=="gaussian":
            self.a=np.random.randn(X.shape[1],self.l)
            self.b=np.random.random()
            h=np.array([np.exp(-self.b*(np.linalg.norm(X-self.a.T[i],axis=1))) for i in range(self.l)]).T
            self.g=h
        self.w=np.linalg.pinv(h)@y
        
    def predict(self,X):
        if self.kind=="sigmoid":
            f_x=X@self.a+self.b
            h=1/(1+np.exp(-f_x))
        elif self.kind=="tanh":
            f_x=X@self.a+self.b
            h=(1-np.exp(-f_x))/(1+np.exp(-f_x))
        elif self.kind=="gaussian":
            h=np.array([np.exp(-self.b*(np.linalg.norm(X-self.a.T[i],axis=1))) for i in range(self.l)]).T
        return h@self.w


In [9]:
class CrossValidation:
    def __init__(self,kind='holdout',seed=2010):
        self.kind=kind
        np.random.seed(seed)
        
    def train_test_split(self,X,y,train_per=False,n_folds=False):
        if self.kind=='holdout':
            train_rows=random.sample(range(0,y.size),train_per*y.size//100)
            train_rows.sort()
            test_rows=[rows for rows in X.index.values if rows not in train_rows]
            
            return X.iloc[train_rows].values,y.iloc[train_rows].values,X.iloc[test_rows].values,y.iloc[test_rows].values
        elif self.kind=='fold':
            
            total=random.sample(range(0,y.size),y.size)
            train_rows=[]
            test_rows=[]
            for i in range(n_folds):
                test_selected_row=total[y.size//n_folds*i:y.size//n_folds*(i+1)]
                test_rows.append(test_selected_row)
                test_rows[i].sort()
                train_rows.append([row for row in total if row not in test_selected_row])
                train_rows[i].sort()
            return train_rows,test_rows

In [10]:
elm=ELM(l=2000)
elm.train(X.values,y.values.reshape(-1,1),kind="gaussian")

In [11]:
cv=CrossValidation(kind='fold')
train_rows,test_rows=cv.train_test_split(X.values,y.values.reshape(-1,1),n_folds=5)

In [12]:
acc=[]
for i in range(len(train_rows)):
    train_X=X.iloc[train_rows[i]].values
    train_y=y.iloc[train_rows[i]].values.reshape(-1,1)
    test_X=X.iloc[test_rows[i]].values
    test_y=y.iloc[test_rows[i]].values.reshape(-1,1)
    elm=ELM(l=500)
    elm.train(train_X,train_y.reshape(-1,1),kind="gaussian")
    y_pred=elm.predict(test_X)
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0
    print(i," Fold")
    print("Confusion Matrix")
    print(confusion_matrix(test_y, y_pred)) 
    print("Accuracy")
    acc.append(accuracy_score(test_y, y_pred)*100)
    print(acc[i]) 
    print('\n')
print("Overall accuracy: ",np.mean(acc))

0  Fold
Confusion Matrix
[[187  14]
 [ 44 184]]
Accuracy
86.48018648018649


1  Fold
Confusion Matrix
[[203  21]
 [ 12 193]]
Accuracy
92.3076923076923


2  Fold
Confusion Matrix
[[200  16]
 [ 37 176]]
Accuracy
87.64568764568764


3  Fold
Confusion Matrix
[[203  19]
 [ 30 177]]
Accuracy
88.57808857808858


4  Fold
Confusion Matrix
[[188  22]
 [ 59 160]]
Accuracy
81.11888111888112


Overall accuracy:  87.22610722610723


# Q7 - Deep NN

In [14]:
import numpy as np
from scipy.io import loadmat
from sklearn.preprocessing import normalize

#Load data, shuffle and normalize
mat_contents = loadmat('/content/drive/MyDrive/2018A8PS1229H_NNFL/Data/data5.mat')
data = mat_contents['x']
np.random.shuffle(data)


def init_data():
    X = np.array(data[ : , :-1], dtype = float)
    y = np.array(data[ : , -1], dtype = int)
    X = normalize(X, axis = 0)
    return X, y

#Convert to one-hot
X, y_ = init_data()
y = np.zeros((len(y_), 2))
for i in range(len(y_)):
    if y_[i]==1:
        y[i,1] = 1.0
    elif y_[i]==0:
        y[i,0] = 1.0

#Hold out method of model evaluation
X_train, y_train = X[ :int(0.7 * len(X))], y[ :int(0.7 * len(X))]
X_val, y_val = X[ int(0.7 * len(X)): ], y[ int(0.7 * len(X)): ]

alpha = 0.5

#Sigmoid activation function
def sigmoid(x, derivative=False):
        if (derivative == True):
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

#Tanh activation function
def tanh(x):
    return np.tanh(x)

In [15]:
#Neural network class
class NeuralNetwork(object):
    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.W = {}
        self.a = {}
        self.b = {}
        
        #Initialize Weights
        for i in range(1, self.num_layers):
            self.W[i] = np.random.randn(self.sizes[i-1], self.sizes[i])
            
        #Initialize biases
        for i in range(1, self.num_layers):
            self.b[i] = np.random.randn(self.sizes[i], 1)
        
        #Initialize activations
        for i in range(1, self.num_layers):
            self.a[i] = np.zeros([self.sizes[i], 1])
        
    #Forward pass to compute scores
    def forward_pass(self, X):
        
        self.a[0] = X
        
        for i in range(1, self.num_layers):
            self.a[i] = sigmoid(np.dot(self.W[i].T, self.a[i-1]) + self.b[i])

        return self.a[self.num_layers-1] 
    
    #Backward pass to update weights
    def backward_pass(self, X, Y, output):
        
        self.d = {}
        self.d_output = (Y - output) * sigmoid(output, derivative=True)
        self.d[self.num_layers-1] = self.d_output
        
        #Derivatives of the layers wrt loss
        for i in range(self.num_layers-1, 1, -1):
            self.d[i-1] = np.dot(self.W[i], self.d[i]) * sigmoid(self.a[i-1], derivative=True)
        
        #Updating weights
        for i in range(1, self.num_layers-1):
            self.W[i] += alpha * np.dot(self.a[i-1], self.d[i].T)
            
        #Updating biases
        for i in range(1, self.num_layers-1):
            self.b[i] += alpha * self.d[i]

    #Training helper function   
    def train(self, X, Y):
        X = np.reshape(X, (len(X), 1))
        output = self.forward_pass(X)
        self.backward_pass(X, Y, output)

    #Get weights    
    def get_W(self):
        return self.W
    
    #Load specified weights
    def load_W(self, W):
        self.W = W

    #Scores computation for given input    
    def get_a(self, x):
        x = np.reshape(x, (len(x), 1))
        self.forward_pass(x)
        return self.a
    
    #Helper function for autoencoder chaining
    def load_a(self, a):
        self.a = a

In [16]:
#Loss function
def calc_loss(NN,x ,y):
    
    loss = 0
    for i in range(len(x)):
        x_ = np.reshape(x[i], (len(x[i]), 1))
        loss += 0.5 / len(x) * np.sum((y[i] - NN.forward_pass(x_)) ** 2)
    
    return loss

In [17]:
#Network initialization
autoencoder1 = NeuralNetwork([72, 60, 72])
autoencoder2 = NeuralNetwork([60,40,60])

#Autoencoder 1 pretraining
for i in range(500):
    for j, row in enumerate(X_train):
        row = np.reshape(row, (72,1))
        autoencoder1.train(row, row)
        
    loss = calc_loss(autoencoder1, X_train, X_train)
    print("Epoch {}, Loss {}".format(i, loss))
    
#Scores computation for autoencoder 1
autoencoder2_input = []

for row in X_train:
    autoencoder2_input.append(autoencoder1.get_a(row)[1])

autoencoder2_input = np.array(autoencoder2_input)


#Autoencoder 2 pretraining
for i in range(500):
    for j, row in enumerate(autoencoder2_input):
        row = np.reshape(row, (60,1))
        autoencoder2.train(row, row)
        
    loss = calc_loss(autoencoder2, autoencoder2_input, autoencoder2_input)
    print("Epoch {}, Loss {}".format(i, loss))

Epoch 0, Loss 317.217409014991
Epoch 1, Loss 313.9464575496261
Epoch 2, Loss 312.5637533011869
Epoch 3, Loss 311.93938692100784
Epoch 4, Loss 311.5867632213122
Epoch 5, Loss 311.29056370252044
Epoch 6, Loss 310.22066870058165
Epoch 7, Loss 309.88686384268647
Epoch 8, Loss 308.76609743084606
Epoch 9, Loss 306.91553860332317
Epoch 10, Loss 306.65355551211553
Epoch 11, Loss 306.4709894812287
Epoch 12, Loss 306.2082289154609
Epoch 13, Loss 306.0855187197632
Epoch 14, Loss 305.9395023683026
Epoch 15, Loss 305.6958815339498
Epoch 16, Loss 305.3952261831466
Epoch 17, Loss 305.33423659636367
Epoch 18, Loss 305.2835280436439
Epoch 19, Loss 305.2385030403021
Epoch 20, Loss 305.20787086417835
Epoch 21, Loss 305.1857807878255
Epoch 22, Loss 305.1662345757563
Epoch 23, Loss 305.14804532897114
Epoch 24, Loss 305.1311110988813
Epoch 25, Loss 305.11567893358335
Epoch 26, Loss 305.10209307456154
Epoch 27, Loss 305.0904293799262
Epoch 28, Loss 305.08038604241455
Epoch 29, Loss 305.07153522742857
Epoch 3

In [18]:
#Inputs to ELM

elm_input = []
for row in autoencoder2_input:
    elm_input.append(autoencoder2.get_a(row)[1])   
elm_input = np.array(elm_input)

#parameters for ELM
elm_neurons = 300
output_neurons = 2
W_elm = np.random.randn(elm_input.shape[1], elm_neurons)

#ELM Training
np.random.seed(1)
elm_input = np.reshape(elm_input, (1503, 40))
H = np.matmul(elm_input, W_elm)
H = tanh(H)
H_inv = np.linalg.pinv(H)
W_final = np.matmul(H_inv, y_train) 

#Testing on validation dataset

#Autoencoder 1 forward pass
layer1_out = []

for i, row in enumerate(X_val):
    act = autoencoder1.get_a(row)[1]
    layer1_out.append(act)
    
layer1_out = np.array(layer1_out)
layer1_out = np.reshape(layer1_out, (645, 60))

#Autoencoder 2 forward pass
layer2_out = []

for i, row in enumerate(layer1_out):
    act = autoencoder2.get_a(row)[1]
    layer2_out.append(act)
    
layer2_out = np.array(layer2_out)
layer2_out = np.reshape(layer2_out, (645, 40))

#ELM forward pass
H_T = np.matmul(layer2_out, W_elm)
H_T = tanh(H_T)
y_pred = np.matmul(H_T, W_final)

TP,TN,FP,FN = 0,0,0,0

for i in range(len(y_pred)):
    
    if np.argmax(y_pred[i]) == 1 and np.argmax(y_val[i]) == 1:
        TP += 1
    elif np.argmax(y_pred[i]) == 0 and np.argmax(y_val[i]) == 0:
        TN += 1
    elif np.argmax(y_pred[i]) == 1 and np.argmax(y_val[i]) == 0:
        FP += 1
    elif np.argmax(y_pred[i]) == 0 and np.argmax(y_val[i]) == 1:
        FN += 1

print(TP, FP)
print(FN, TN)

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("accuracy = ", accuracy, "sensitivity = ", sensitivity, "specificity = ", specificity)

263 43
63 276
accuracy =  0.8356589147286821 sensitivity =  0.8067484662576687 specificity =  0.8652037617554859
